# MAVISIM Tutorial

This is an example notebook showing you how to create a stellar field using an example input file and spatially variable low- and high-order PSFs.

# Imports

In [ ]:
# Standard
import numpy as np
import os

# Astropy
from astropy.io import fits, ascii
from astropy.table import Table

# Plotting
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

# Project Specific
import input_parameters as input_par

from mavisim.Source import Source
from mavisim.AOGaussGrid import AOGaussGrid
from mavisim.SeeingGrid import SeeingGrid
from mavisim.addnoise import add_all_noise
from mavisim.InputCoo import InputCoo

# Load the input data and specify a exposure time

We're loading an N-body catalogue of NGC 3201 with an embedded 1500 intermediate mass black hole in the centre. 



In [ ]:
# Nbody input plus chosen exposure time
ngc3201 = input_par.input_file

# Exposure time in seconds
exp_time = 30

# To create the source object we need to specify two things:

## 1. Tip-tilt Jitter Map Variability (on or off) with `tt_var` and
## 2. Specify the static distortion (on or off) with the `static_dist` parameter

In [ ]:
# We're going to run the simulation with the tip-tilt residual being spacially variable and including static distortion

# Some information about the specifications are printed when Source is called
source = Source(input_par, ngc3201, exp_time, static_dist=True, tt_var=True).main()

# Create the AO and Gaussian Fields, specify one thing:

## 1. High-Order PSF Spatial Variability (on or off) with `psf_var`

In [ ]:
# Create the ao_field (which will be our final field in the case of the big EtE PSxF)
(ao_field,gauss_field) = AOGaussGrid(input_par, source, fv_psf=True).main()

# Create the Seeing FIeld

In [ ]:
# Create the seeing field
seeing_field = SeeingGrid(input_par, gauss_field).main()

# Make the Noise-Free Image

In [ ]:
# Add the two fields to make the image
image = ao_field + seeing_field

# Create the Final Image

In [ ]:
# Add shot noise, read noise and convert from electrons to ADU
final_image = add_all_noise(input_par, image, source.meta["exp_time"])

## Optional Code: Create a Catalogue of the Input Stellar Positions

In [ ]:
# Create the input catalogue
input_coo = InputCoo(input_par, source).main()

## Optional Code: Save the Image as a Fits File for use with DAOPHOT

In [ ]:
image_final_noise = np.array(final_image, dtype="float32")
hdu = fits.PrimaryHDU(image_final_noise)
hdul = fits.HDUList([hdu])
hdul.writeto("testimage.fits", overwrite=True)

# Save the input posiions
ascii.write(input_coo, "testpos.coo", overwrite=True)

## Optional Code: Visualise the Tip-Tilt Map

In [ ]:
# This is the tip-tilt map we'll be using as specified in the input_parameters.py file

# This is the best map, representing the ideal constellation geometry and characterstics
print (input_par.tt_residual_map)

In [ ]:
def plot_ngs(star_locs, mag, axis):
    x = star_locs[0] * np.cos(star_locs[1] * np.pi/180)
    y = star_locs[0] * np.sin(star_locs[1] * np.pi/180)
    
    axis.scatter(x, y, marker="*", s=8000*(1/mag), c="r", zorder=2, alpha=0.9)

In [ ]:
def plot_jitter(jitter_map, axis, coeff):
    xs = jitter_map[0].data[0]
    ys = jitter_map[0].data[1]
    
    for num in range(0, 11):
        y = ys[num][0]

        for num2 in range(0, 11):
            x = xs[num, num2]
            
            # Can keep track of the kernel parameters if you'd like!
            major_ax = jitter_map[0].data[2][num, num2]
            minor_ax = jitter_map[0].data[3][num, num2]
            theta = jitter_map[0].data[4][num, num2]
            
            # Plot the tip-tilt kernel as ellipses
            ell = Ellipse(xy=(x, y), width=2*coeff*major_ax, height=2*coeff*minor_ax, angle=theta)
            ell.set_facecolor("black")
            ell.set_alpha(0.75)
            
            axis.add_artist(ell)

In [ ]:
jitter_map1_ngs_locs = [[10, 0], [10, 120], [10, 240]]
jitter_map1_ngs_mags = [15, 15, 15]

fig, ax1 = plt.subplots(figsize=(6, 6))

ax1.minorticks_on()
ax1.tick_params(which='major', length=6, width=1, direction='in', top=True, right=True)
ax1.tick_params(which='minor', length=2, width=1, direction='in', top=True, right=True)

plot_jitter(input_par.tt_jitter, ax1, coeff=100/1e3)

for (loc, mag) in zip(jitter_map1_ngs_locs, jitter_map1_ngs_mags):
    plot_ngs(loc, mag, ax1)

ax1.set_xlim(-20, 20)
ax1.set_ylim(-20, 20)
ax1.set_xlabel('X [arcsec]')
ax1.set_ylabel('Y [arcsec]')

ax1.hlines(y=0, xmin=-20, xmax=20, color="k", linestyle=":", zorder=1)
ax1.vlines(x=0, ymin=-20, ymax=20, color="k", linestyle=":", zorder=1)

plt.show()
